# [Advent of Code 2020 Day ?]()

?

## Initial setup

In [55]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Input Parsing

In [56]:
def parse_input(filename: str) -> Any:

    gen = yield_line(filename)

    result = []
    rules = []

    while (next_line := next(gen)) != "":
        rules.append(parse(f"(.+): (\d+)\-(\d+)\ or (\d+)\-(\d+)", next_line))

    assert next(gen).startswith("your ticket:")

    your_ticket = intsep(next(gen), ",")

    assert next(gen) == ""
    assert next(gen).startswith("nearby tickets:")

    other_tickets = []

    for line in gen:
        other_tickets.append(intsep(line, ","))

    return rules, your_ticket, other_tickets

## Part 1
Lorem ipsum

In [57]:
def part_one(data: Any) -> int | str:
    rules, your_ticket, other_tickets = data

    def validate_rule(idx: int, value: int):
        name, min_a, max_a, min_b, max_b = rules[idx]
        return int(min_a) <= int(value) <= int(max_a) or int(min_b) <= int(value) <= int(max_b)

    def validate_rules(value: int):
        for i in range(len(rules)):
            if validate_rule(i, value):
                return True
        return False

    result = 0

    for ticket in other_tickets:
        for idx, value in enumerate(ticket):
            if not validate_rules(value):
                result += value

    return result

In [58]:
%%ipytest
def test_part_one():
    assert part_one(parse_input("example1")) == 71
    assert part_one(parse_input("input")) == 27850

.                                                                                            [100%]
1 passed in 0.01s


## Part 2
Lorem ipsum

In [59]:
def part_two(data: Any) -> int | str:
    return 0x3f3f3f3f + 2

In [60]:
%%ipytest
def test_part_two():
    assert part_two(parse_input("example1")) == 0x3f3f3f3f + 2
    assert part_two(parse_input("input")) == 0x3f3f3f3f + 2

.                                                                                            [100%]
1 passed in 0.01s
